In [1]:
import pandas as pd
import numpy as np

### 1. Capitalization


#### Load tag_movie data

In [2]:
# This block counts how many times a tag has been applied; 
# Used for weighting user tags

import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_dev")
try:
    with connection.cursor() as cursor:
            sql = "SELECT * FROM tag_movie"
            cursor.execute(sql)
            result = cursor.fetchall()
            
finally: 
    connection.close()



In [3]:
movie_tag_app = pd.DataFrame(list(result), columns=['movieId','tag','numApps','numPositive','numNeutral','numNegative','numDownvotes','score'])
movie_tag_app.drop(["numPositive","numNeutral","numNegative","numDownvotes"],axis=1,inplace=True)

In [4]:
movie_tag_app_agg = movie_tag_app.groupby("tag").sum().reset_index()

In [5]:
movie_tag_app_agg

,tag,movieId,numApps,score
0,!950's Superman TV show,47950,1,-4.0
1,!David O. Russell,2890,1,-8.0
2,!George Clooney,2890,1,-9.0
3,!George Lucas,260,1,-9.0
4,"""A Christmas Carol""",4023,1,0.0
5,"""A MÃ£o-de-Deus""",5294,1,-1.0
6,"""American"" French",4014,1,-1.0
7,"""Duck and cover!""",8049,2,-1.0
8,"""Ghost for adults""",5034,1,0.0
9,"""I need a computer.""",172,1,-1.0


In [6]:
tag_lowercase_list = []
tag_lowercase_dup_dict = {}

for index, row in movie_tag_app_agg.iterrows():
    tag = row[0]
    
    tag_lower = tag.lower()
    tag_lowercase_list.append(tag_lower)
    if tag_lower not in tag_lowercase_dup_dict:
        tag_lowercase_dup_dict[tag_lower] = []
        tag_lowercase_dup_dict[tag_lower].append(tag)
    else:
        tag_lowercase_dup_dict[tag_lower].append(tag)
        
movie_tag_app_agg['tag_lowercase'] = tag_lowercase_list


In [50]:
tag_lowercase_dup_dict['"bad cia"-- too simplistic']

['"bad CIA"-- too simplistic', '"bad cia"-- too simplistic']

In [7]:
tag_lowercase_dup_list = []


for index, row in movie_tag_app_agg.iterrows():
    tag = row[0]
    tag_lowercase = row[4]
    temp = ""
    for item in tag_lowercase_dup_dict[tag_lowercase]:
       # print(tag_lowercase)
        #print(item)
        if item != tag:
            temp += item + ";"
    if temp != "":
        tag_lowercase_dup_list.append(temp)
    else:
        tag_lowercase_dup_list.append(None)
    
movie_tag_app_agg["tag_lowercase_dups"] = tag_lowercase_dup_list


In [8]:
# Find capitalization duplicates

movie_tag_app_agg.sort_values(by="score",ascending=False,inplace=True)
movie_tag_app_agg.sort_values(by="tag_lowercase",inplace=True)
#movie_tag_app_agg[1000:1100]

In [19]:
#movie_tag_app_agg.drop_duplicates("tag_lowercase",keep="first")
tag_no_capitalization_dup = movie_tag_app_agg.drop_duplicates("tag_lowercase",keep="first")
tag_no_capitalization_dup

,tag,movieId,numApps,score,tag_lowercase,tag_lowercase_dups
0,!950's Superman TV show,47950,1,-4.0,!950's superman tv show,None
1,!David O. Russell,2890,1,-8.0,!david o. russell,None
2,!George Clooney,2890,1,-9.0,!george clooney,None
3,!George Lucas,260,1,-9.0,!george lucas,None
4,"""A Christmas Carol""",4023,1,0.0,"""a christmas carol""",None
5,"""A MÃ£o-de-Deus""",5294,1,-1.0,"""a mã£o-de-deus""",None
6,"""American"" French",4014,1,-1.0,"""american"" french",None
23,"""artsy""",4552,2,2.0,"""artsy""",None
24,"""bad CIA""-- too simplistic",74545,1,0.0,"""bad cia""-- too simplistic","""bad cia""-- too simplistic;"
26,"""based on true events""",58201,2,2.0,"""based on true events""",None


In [20]:
tag_no_capitalization_dup.drop(["numApps","tag_lowercase"],axis=1,inplace=True)

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [21]:
tag_no_capitalization_dup = tag_no_capitalization_dup.reset_index()

In [22]:
tag_no_capitalization_dup.drop(["index"],axis=1,inplace=True)
tag_no_capitalization_dup

,tag,movieId,score,tag_lowercase_dups
0,!950's Superman TV show,47950,-4.0,None
1,!David O. Russell,2890,-8.0,None
2,!George Clooney,2890,-9.0,None
3,!George Lucas,260,-9.0,None
4,"""A Christmas Carol""",4023,0.0,None
5,"""A MÃ£o-de-Deus""",5294,-1.0,None
6,"""American"" French",4014,-1.0,None
7,"""artsy""",4552,2.0,None
8,"""bad CIA""-- too simplistic",74545,0.0,"""bad cia""-- too simplistic;"
9,"""based on true events""",58201,2.0,None


In [23]:
tag_no_capitalization_dup.to_csv("tag_no_capitalization_dup.csv")

In [ ]:
## Load tag_no_capitalization_duplicates

# tag_no_capitalization_dup = pd.read_csv("tag_no_capitalization_dup.csv")


### 2. Process dash and spaces

In [24]:
tag_no_capitalization_dup = pd.read_csv("tag_no_capitalization_dup.csv",index_col=0)
tag_no_capitalization_dup

,tag,movieId,score,tag_lowercase_dups
0,!950's Superman TV show,47950,-4.0,NaN
1,!David O. Russell,2890,-8.0,NaN
2,!George Clooney,2890,-9.0,NaN
3,!George Lucas,260,-9.0,NaN
4,"""A Christmas Carol""",4023,0.0,NaN
5,"""A MÃ£o-de-Deus""",5294,-1.0,NaN
6,"""American"" French",4014,-1.0,NaN
7,"""artsy""",4552,2.0,NaN
8,"""bad CIA""-- too simplistic",74545,0.0,"""bad cia""-- too simplistic;"
9,"""based on true events""",58201,2.0,NaN


#### Strip punctuations

In [77]:
import string
table = str.maketrans('','',string.punctuation)

In [79]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [82]:
table = str.maketrans('', '', string.punctuation)

In [93]:
tag_strip_punc = []
tag_punctuation_dup_dict = {}

# Strip punctuation and store tag - tag_no_punctuation pairs
for index, row in tag_no_capitalization_dup.iterrows():
    tag = row[0]
    stripped_tag = tag.translate(table)
    tag_strip_punc.append(stripped_tag)
    if stripped_tag not in tag_punctuation_dup_dict:
        tag_punctuation_dup_dict[stripped_tag] = []
        tag_punctuation_dup_dict[stripped_tag].append(tag)
    else:
        tag_punctuation_dup_dict[stripped_tag].append(tag)

{'950s Superman TV show': ["!950's Superman TV show"],
 'David O Russell': ['!David O. Russell', 'David O. Russell'],
 'George Clooney': ['!George Clooney', 'George Clooney'],
 'George Lucas': ['!George Lucas', 'George Lucas', 'George Lucas.'],
 'A Christmas Carol': ['"A Christmas Carol"', 'A Christmas Carol'],
 'A MÃ£odeDeus': ['"A MÃ£o-de-Deus"'],
 'American French': ['"American" French'],
 'artsy': ['"artsy"', 'artsy'],
 'bad CIA too simplistic': ['"bad CIA"-- too simplistic'],
 'based on true events': ['"based on true events"', 'based on true events'],
 'bitchy': ['"bitchy"', 'bitchy'],
 'comédie': ['"comédie"', 'comédie'],
 'damn dirty apes': ['"damn dirty apes"', 'damn dirty apes'],
 'Duck and cover': ['"Duck and cover!"'],
 'eccentric genius': ['"eccentric genius"'],
 'found footage': ['"found footage"'],
 'Ghost for adults': ['"Ghost for adults"'],
 'hero is murderer': ['"hero" is murderer'],
 'I need a computer': ['"I need a computer."'],
 'inspired by a book': ['"inspired" by

In [164]:
tag_no_capitalization_dup['tag_strip_punctuation'] = tag_strip_punc
tag_no_cap_punc_dup = tag_no_capitalization_dup

In [165]:
tag_punct_dup_list = []


for index, row in tag_no_cap_punc_dup.iterrows():
    tag = row[0]
    tag_no_punc = row[4]
    temp = ""
    for item in tag_punctuation_dup_dict[tag_no_punc]:
       # print(tag_lowercase)
        #print(item)
        if item != tag:
            temp += item + ";"
    if temp != "":
        tag_punct_dup_list.append(temp)
    else:
        tag_punct_dup_list.append(None)
    
tag_no_cap_punc_dup["tag_punc_dups"] = tag_punct_dup_list


In [176]:
## Explain sort by scores

tag_no_cap_punc_dup.sort_values(by=["tag_strip_punctuation","score"],ascending=False,inplace=True)

In [178]:
tag_no_cap_punc_dup[2000:2100]

,tag,movieId,score,tag_lowercase_dups,tag_strip_punctuation,tag_punc_dups
62491,visual effect,172593,2.0,Visual effect;,visual effect,None
62490,visual design,45672,0.0,NaN,visual design,None
62489,visual comedy,103341,1.0,NaN,visual comedy,None
62488,visual arts,851,1.0,NaN,visual arts,None
62487,visual appealing,411920,7.0,Visual appealing;,visual appealing,None
62485,visual apealing,168064,1.0,Visual Apealing;,visual apealing,None
62484,visual aesthetic,3863,1.0,NaN,visual aesthetic,None
62483,visual,2075983,123.0,NaN,visual,None
62481,vistas,118900,1.0,NaN,vistas,None
62480,vista vision,8289,-3.0,NaN,vista vision,None


In [179]:
### To demonstrate some tags that will be dropped
#tag_no_cap_punc_dup_small = tag_no_cap_punc_dup[tag_no_cap_punc_dup['score']>=2].copy()
#tag_no_cap_punc_dup_small.sort_values(by="tag_strip_punctuation")


### Drop tag duplicates with punctuation:
# tag_no_cap_punc_dup.sort_values(by="tag_strip_punctuation")
# tag_no_cap_punc_dup.drop_duplicates("tag_punc_dups",keep=first)

tag_no_cap_punc_dup.drop_duplicates("tag_strip_punctuation",keep="first",inplace=True)


In [188]:
# Using violence and violence. as an exmpale. violence. is dropped. 
# tag_no_cap_punc_dup[2050:2100]

,tag,movieId,score,tag_lowercase_dups,tag_strip_punctuation,tag_punc_dups
62391,violence in america,22301,10.0,NaN,violence in america,None
62390,violence fiction,7438,-1.0,NaN,violence fiction,None
62389,"violence between wife and husband, who loved e...",33679,1.0,NaN,violence between wife and husband who loved ea...,None
62388,violence against women,331577,10.0,Violence against women;,violence against women,None
62387,violence against tyres,81132,0.0,NaN,violence against tyres,None
62386,violence against children,50,0.0,NaN,violence against children,None
62385,violence - too much for me!!,68157,-1.0,NaN,violence too much for me,None
62384,violence - Jennifer Connelly!!!!,3949,-6.0,NaN,violence Jennifer Connelly,None
62383,violence,88789624,2808.0,Violence;,violence,violence.;
62382,violante placido,80126,1.0,NaN,violante placido,None


In [190]:
tag_no_cap_punc_dup.drop("tag_strip_punctuation",axis=1,inplace=True)
tag_no_cap_punc_dup.to_csv("tag_no_cap_punc_dup.csv",index=False)

In [ ]:
## Load tags without capitalization / punctuation duplicates


# tag_no_cap_punc_dup = pd.read_csv("tag_no_cap_punc_dup.csv",index=False)


### 3. Stemming


Note: stemming sometimes produce poor results

In [71]:
# Read from tag_no_cap_punc_dup
tag_no_cap_punc_dup = pd.read_csv("tag_no_cap_punc_dup.csv")

In [72]:
tag_list = tag_no_cap_punc_dup["tag"].tolist()

In [73]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [74]:
ps = PorterStemmer()

In [75]:
stem_result = {}
dup_words = []

for w in tag_list:
    #print(w) 
    curstem = ps.stem(w)
    if curstem in stem_result:
        stem_result[curstem].append(w)
    else:
        stem_result[curstem] = []
        stem_result[curstem].append(w)

#### Add tag stems as a new column to tag_score

In [76]:
tag_stems = []

for w in tag_list:
    curstem = ps.stem(w)
    tag_stems.append(curstem)

 

In [77]:
tag_no_cap_punc_dup["tag_stem"] = tag_stems

In [78]:
tag_no_cap_punc_dup.head()

,tag,movieId,score,tag_lowercase_dups,tag_punc_dups,tag_stem
0,카운트다운,144340,1.0,NaN,NaN,카운트다운
1,魔鬼司令,6664,1.0,NaN,NaN,魔鬼司令
2,难看,111781,1.0,NaN,NaN,难看
3,臥底,148940,1.0,NaN,NaN,臥底
4,科幻 冒险,260,1.0,NaN,NaN,科幻 冒险


In [79]:
## For each tag, find tags with the same stem. 
## Add 'tag_same_stem' as a new column to tag_score_sorted.

tag_same_stem = []

for index, row in tag_no_cap_punc_dup.iterrows():
    tag = row[0]
    stem = row[5]
    temp = ''
    if stem in stem_result:
        #same_stem_list = stem_result[stem]
        for t in stem_result[stem]:
            if t != tag:
                #temp.append(k)
                temp += t+";"
    if temp != '':
        tag_same_stem.append(temp)
    else:
        tag_same_stem.append(None)



In [80]:
tag_no_cap_punc_stem_dup = tag_no_cap_punc_dup
tag_no_cap_punc_stem_dup["tags_same_stem"] = tag_same_stem

#### Potentially using stem to drop duplicates

In [82]:
tag_no_cap_punc_stem_dup.sort_values(by=["tag_stem","score"],ascending=False,inplace=True)


In [230]:
## An example of what happens if we drop tag_same_stem_duplicates

# tag_no_cap_punc_stem_dup_demo = tag_no_cap_punc_stem_dup[50000:50100]
# tag_no_cap_punc_stem_dup_demo

## Drop Duplicates
# tag_no_cap_punc_stem_dup_demo.drop_duplicates("tag_stem",keep="first")

,tag,movieId,score,tag_lowercase_dups,tag_punc_dups,tag_stem,tags_same_stem
32094,devil,7986920,156.0,Devil;,NaN,devil,None
60023,Devid Striesow,58303,-1.0,NaN,NaN,devid striesow,None
32095,developmental disorder,84234,1.0,NaN,NaN,developmental disord,None
32096,developmental disability,134111,1.0,NaN,NaN,developmental dis,None
60024,Development through dialogue,3906,0.0,NaN,NaN,development through dialogu,None
32097,development hell,309055,2.0,NaN,NaN,development hel,None
60025,Development friendship,103141,0.0,NaN,NaN,development friendship,None
60026,Developing Doomed Characters,165105,1.0,NaN,NaN,developing doomed charact,None
32098,development,513440,6.0,NaN,NaN,develop,developer;
60027,Devdas,136495,1.0,NaN,NaN,devda,None


Note: stemming sometimes produce poor results.
For example, determination and determinism both have the stem: determin, but they have different meanings

In [ ]:
# tag_no_cap_punc_stem_dup_demo.drop_duplicates("tag_stem",keep="first")

### 4. Combine capitalization dups and punctuation dups into one column


In [83]:
tag_no_cap_punc_stem_dup.head()

,tag,movieId,score,tag_lowercase_dups,tag_punc_dups,tag_stem,tags_same_stem
0,카운트다운,144340,1.0,NaN,NaN,카운트다운,None
1,魔鬼司令,6664,1.0,NaN,NaN,魔鬼司令,None
2,难看,111781,1.0,NaN,NaN,难看,None
3,臥底,148940,1.0,NaN,NaN,臥底,None
4,科幻 冒险,260,1.0,NaN,NaN,科幻 冒险,None


In [84]:
# Fill nan with ""

tag_no_cap_punc_stem_temp = tag_no_cap_punc_stem_dup.fillna("")

In [85]:
dup_list = []

for index, row in tag_no_cap_punc_stem_temp.iterrows():
    tag = row[0]
    capitalization_dup = row[3]
    
    punctuation_dup = row[4]
#     print(capitalization_dup)
#     print(punctuation_dup)
    if (capitalization_dup != "" and punctuation_dup != ""):
        dups = str(capitalization_dup)+str(punctuation_dup)
    elif capitalization_dup != "":
        dups = capitalization_dup
    elif punctuation_dup != "":
        dups = punctuation_dup
    else:
        dups = None
    dup_list.append(dups)


In [ ]:
tag_no_cap_punc_stem_dup['duplicates']=dup_list

In [88]:
tag_no_cap_punc_stem_dup_small = tag_no_cap_punc_stem_dup.drop(["tag_lowercase_dups","tag_punc_dups"],axis=1)


In [93]:
tag_no_cap_punc_stem_dup_small.to_csv("tag_no_cap_punc_stem_dup.csv",index=False)

In [28]:
# Load dataset
tag_no_cap_punc_stem_dup_small = pd.read_csv("tag_no_cap_punc_stem_dup.csv")
tag_no_cap_punc_stem_dup_small

,tag,movieId,score,tag_stem,tags_same_stem,duplicates
0,sci-fi,30393804,7842.0,sci-fi,NaN,Sci-Fi;Sci-fi;#scifi;+scifi;
1,atmospheric,22351413,5399.0,atmospher,Atmosphere;,ATMOSPHERIC;Atmospheric;
2,surreal,30372701,4581.0,surreal,surrealism;,Surreal;
3,twist ending,14940576,4102.0,twist end,twist endings;twist end;,Twist Ending;Twist ending;
4,visually appealing,20032372,3866.0,visually app,NaN,Visually Appealing;Visually appealing;
5,comedy,107137337,3775.0,comedi,comedie;,COMEDY;Comedy;#comedy;
6,psychology,13220258,3209.0,psycholog,psychological;psychologic;,Psychology;
7,stylized,12154343,3097.0,styliz,NaN,Stylized;
8,based on a book,64330991,2999.0,based on a book,NaN,Based on a Book;Based on a book;
9,fantasy,34123204,2811.0,fantasi,fantasies;,Fantasy;#fantasy;
